In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import torch
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50
from multiprocessing import freeze_support
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F
import torchvision.models as models
from PIL import Image
import os
import glob
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.preprocessing import MinMaxScaler
import pandas as pd

min_max_scaler = MinMaxScaler()

In [ ]:
class MyDataSet(Dataset):
    def __init__(self, root, image_size, transform=None):
        self.images_path = []
        self.images_class = []
        target_folder = os.path.join(root, "*.png")
        
        for file_path in glob.glob(target_folder):
            self.images_path.append(file_path)
            image_name = os.path.basename(file_path)
            first_part = image_name.split("_")[0]
            if(first_part == '0'):
                self.images_class.append(0)
            elif(first_part == '1'):
                self.images_class.append(1)
            elif(first_part == '2'):
                self.images_class.append(2)
            elif(first_part == '3'):
                self.images_class.append(3)
            elif(first_part == '4'):
                self.images_class.append(4)
            elif(first_part == '5'):
                self.images_class.append(5)
            elif(first_part == '6'):
                self.images_class.append(6)
            elif(first_part == '7'):
                self.images_class.append(7)
            elif(first_part == '8'):
                self.images_class.append(8)
            elif(first_part == '9'):
                self.images_class.append(9)
            else:
                print("图片读取错误")
            
            self.height_pd = pd.DataFrame(self.images_class)

            self.height_pd.columns = ['label'] 

            
            mean_height_list = [110.0, 133.0, 165.0, 175.0, 175.0, 176.0, 186.0, 160.0, 165.0, 163.0]  
            def setting_height(row):
                height = sorted(np.random.normal(mean_height_list[int(row)], 0.5,size=1))[0] 
                return np.round(height, 2)
                # height = mean_height_list[int(row)]
                # return height
                
            self.height_pd['height'] = self.height_pd['label'].apply(setting_height)
            print(self.height_pd['height'])
            # print('-------------------')
            # print(self.height_pd.sample(5))
            # print('-------------------')

            min_max_scaler.fit(self.height_pd)
            self.height = min_max_scaler.transform(self.height_pd)[:, 1]
            self.height_label = min_max_scaler.transform(self.height_pd)[:, 0]  
            
            self.transform = transform
            self.image_size = image_size

    
    def __getitem__(self, index):
        
        image = Image.open(self.images_path[index])
        if image is None:
            print("图像为空")
        label = self.height[index]
        if self.transform is not None:
            image = self.transform(image)
        transf = transforms.Compose([transforms.RandomRotation(degrees=15),
                                     transforms.RandomResizedCrop(size=self.image_size, scale=(0.8, 1.0)),
                                     transforms.ToTensor(),
                                    #  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                     ])
        image =  transf(image)
        label = torch.tensor(label)
        return image, label
    
    def __len__(self):
        return len(self.images_path)

In [ ]:
def load_data(data_path, batch_size, num_workers, train_ratio=0.7, val_ratio=0.2, image_size=224):
    dataset = MyDataSet(data_path, image_size, transform=None)

    num_data = len(dataset)
    num_train = int(num_data * train_ratio)
    num_val = int(num_data * val_ratio)
    num_test = num_data - num_train - num_val

    train_dataset, val_dataset, test_dataset = random_split(dataset, [num_train, num_val, num_test])

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers)

    return train_dataloader, val_dataloader, test_dataloader

In [ ]:
train_loader, val_loader, test_loader = load_data('/content/gdrive/MyDrive/fusion', 32, 4)